# Train with multiple models then compare the results. Use the best scored model for each county.

* Tensorflow LSTM
* Tensorflow Dense
* Linear Regression
* XGBRegression
* Same value

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from xgboost import XGBRegressor

import multiprocessing as mp
from functools import partial
from helper_functions import *

## Load data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv').sort_values(by=['cfips', 'first_day_of_month']).reset_index().drop('index', axis=1)
results = pd.read_csv('lstm_sv_result_with_category.csv')
d = pd.read_csv('add_XGBR_results.csv')

In [3]:
test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1001_2022-12-01,1001,2022-12-01
2,1001_2023-01-01,1001,2023-01-01
3,1001_2023-02-01,1001,2023-02-01
4,1001_2023-03-01,1001,2023-03-01
...,...,...,...
25075,56045_2023-02-01,56045,2023-02-01
25076,56045_2023-03-01,56045,2023-03-01
25077,56045_2023-04-01,56045,2023-04-01
25078,56045_2023-05-01,56045,2023-05-01


In [4]:
results

,Country,LSTM_W9_H8,SV_MAPE,Active,category
0,1001,0.732748,1.835232,1417,lstm
1,1003,0.747286,5.230218,13401,lstm
2,1005,1.273616,1.101416,239,SV
3,1007,9.315642,3.574082,220,SV
4,1009,0.766072,2.116541,789,lstm
...,...,...,...,...,...
3130,56037,2.913013,2.450784,927,SV
3131,56039,0.354971,1.586990,4875,lstm
3132,56041,4.173564,1.381605,574,SV
3133,56043,0.819441,3.608545,183,lstm


In [6]:
result_df = pd.merge(results, d, how='inner', on='Country')
result_df = result_df[['Country', 'LSTM_W9_H8', 'SV_MAPE_x', 'LR_MAPE', 'XGBR_MAPE', 'Active']]
result_df

,Country,LSTM_W9_H8,SV_MAPE_x,LR_MAPE,XGBR_MAPE,Active
0,1001,0.732748,1.835232,0.992271,2.257356,1417
1,1003,0.747286,5.230218,4.317589,93.943110,13401
2,1005,1.273616,1.101416,1.833943,0.912357,239
3,1007,9.315642,3.574082,4.259990,4.514549,220
4,1009,0.766072,2.116541,1.285277,2.398140,789
...,...,...,...,...,...,...
3130,56037,2.913013,2.450784,3.550418,3.110915,927
3131,56039,0.354971,1.586990,2.390681,98.065660,4875
3132,56041,4.173564,1.381605,3.925458,87.435030,574
3133,56043,0.819441,3.608545,2.219479,3.603001,183


In [10]:
result_df.loc[result_df.Active > 20000]

,Country,LSTM_W9_H8,SV_MAPE_x,LR_MAPE,XGBR_MAPE,Active
36,1073,0.613028,1.341826,0.995984,92.237495,32299
101,4013,2.189596,0.809519,0.806137,96.596880,489925
104,4019,0.732151,2.817447,3.065744,92.449615,52908
184,6001,0.820624,3.164619,3.110604,95.536095,143173
190,6013,0.787492,2.597808,3.238149,94.483826,78364
...,...,...,...,...,...,...
2982,53073,1.193863,1.817280,1.348129,96.015430,22305
3052,55025,3.761491,6.157105,5.285632,94.044890,38503
3080,55079,1.271011,4.885237,5.516419,90.565180,36369
3107,55133,0.928716,3.007539,3.937911,93.770150,24607


## Dense with multiple variable

In [6]:
tf_dense_results = d[['Country', 'tf_dense_MAPE']]
tf_dense_results.head()

,Country,tf_dense_MAPE
0,1001,1.504614
1,1003,4.143341
2,1005,1.395184
3,1007,5.228748
4,1009,1.442244


In [12]:
%%time
if __name__ == '__main__':
    HORIZON = 8
    WINDOW_SIZE = 9

    train = pd.read_csv('train.csv')
    IDS = train.cfips.unique()
    #random_ids = IDS[:10]
    
    window_size = WINDOW_SIZE
    horizon = HORIZON
    epoch = 500
    split_size = 6
    
    cfips_list = []
    mapes = []
    density_values = []
    active_size = []
    forecasts = []

    pool = mp.Pool(16)

    func = partial(train_get_results_multi_variables_dense, train, window_size, horizon, epoch, split_size)
    results = pool.map(func, IDS)
    pool.close()
    pool.join()
    
    # Create DataFrame
    for i in range(len(results)):
        c = results[i][0]
        cfips_list.append(c)
        last_density = results[i][1]
        density_values.append(last_density)
        last_active = results[i][2]
        active_size.append(last_active)
        mape = results[i][3]
        mapes.append(mape)
        preds = results[i][4]
        x = np.array(preds).reshape((-1))
        preds = x.astype(np.float64)
        forecasts.append(preds)
    
    dense_result_df = pd.DataFrame({'Country': cfips_list,
                                   'MAPE': mapes,
                                   "Density": density_values,
                                   'Active': active_size,
                                   'forecast': forecasts})

CPU times: total: 6.08 s
Wall time: 17min 41s


In [13]:
dense_result_df.to_csv("dense_W9_H8_results.csv", index=False)

### Compare with old tf_dense_result and new one with W9_H8 multivariable set
### Add Dense result to result_df.csv

so I will have the results on tf_LSTM, tf_Dense, LR, XGBR, and SV. I will categorize the model then create the function to use the model on each County. 

In [18]:
tf_dense_results = pd.read_csv("dense_W9_H8_results.csv")
tf_dense_results['dense_W9_H8_MV'] = tf_dense_results['MAPE']
all_results = pd.merge(result_df, tf_dense_results, how='inner', on='Country')
all_results.drop(['MAPE', 'Density', 'forecast', 'Active_y'],axis=1, inplace=True)
all_results

,Country,LSTM_W9_H8,SV_MAPE_x,LR_MAPE,XGBR_MAPE,Active_x,dense_W9_H8_MV
0,1001,0.732748,1.835232,0.992271,2.257356,1417,0.724775
1,1003,0.747286,5.230218,4.317589,93.943110,13401,2.480568
2,1005,1.273616,1.101416,1.833943,0.912357,239,4.606141
3,1007,9.315642,3.574082,4.259990,4.514549,220,24.276476
4,1009,0.766072,2.116541,1.285277,2.398140,789,1.445454
...,...,...,...,...,...,...,...
3130,56037,2.913013,2.450784,3.550418,3.110915,927,9.501690
3131,56039,0.354971,1.586990,2.390681,98.065660,4875,3.563838
3132,56041,4.173564,1.381605,3.925458,87.435030,574,9.976927
3133,56043,0.819441,3.608545,2.219479,3.603001,183,6.841048


In [19]:
all_results.describe()

,Country,LSTM_W9_H8,SV_MAPE_x,LR_MAPE,XGBR_MAPE,Active_x,dense_W9_H8_MV
count,3135.000000,3135.000000,3135.000000,3.135000e+03,3135.000000,3.135000e+03,3.135000e+03
mean,30376.037640,57.733185,3.909992,8.927551e+03,116.271379,6.604037e+03,1.509228e+04
std,15145.862593,3113.792218,21.621562,4.995858e+05,4497.496999,3.353639e+04,8.445913e+05
min,1001.000000,0.164285,0.000000,2.424637e-01,0.020653,0.000000e+00,2.211540e-01
25%,18178.000000,0.854867,1.394263,1.664201e+00,2.300055,1.455000e+02,2.738610e+00
50%,29173.000000,1.365231,2.353172,2.748950e+00,5.577039,4.890000e+02,4.995527e+00
75%,45076.000000,2.304901,3.993313,4.815689e+00,89.062245,2.139500e+03,8.907000e+00
max,56045.000000,174346.520000,1183.264800,2.797235e+07,251844.530000,1.137607e+06,4.728958e+07


### Label categories

In [22]:
best_score = all_results.min(axis='columns')

cat = []
for i, m in enumerate(best_score):
    if all_results['Active_x'].iloc[i] < 20000:
        category = 'SV'
    elif m == all_results['LSTM_W9_H8'].iloc[i]:
        category = 'LSTM'
    elif m == all_results['SV_MAPE_x'].iloc[i]:
        category = 'SV'
    elif m == all_results['LR_MAPE'].iloc[i]:
        category = 'LR'
    elif m == all_results['XGBR_MAPE'].iloc[i]:
        category = 'XGBR'
    elif m == all_results['dense_W9_H8_MV'].iloc[i]:
        category = 'Dense_MV'
    cat.append(category)
all_results['category'] = cat

C:\Users\Masa\AppData\Local\Temp\ipykernel_14140\3473298627.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  best_score = all_results.min(axis='columns')


In [25]:
all_results.to_csv('all_results_with_category.csv', index=False)

In [23]:
all_results['category'].value_counts()

SV          2972
LSTM         140
Dense_MV      12
LR            11
Name: category, dtype: int64